In [49]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [50]:
path='~/Coursera/Data/Data_final.csv'
df = pd.read_csv(path)
df.head()

,Quartiers,villes,Latitude,Longitude
0,Quartier Saint-Germain-l'Auxerrois,Paris,48.860562,2.336489
1,Les Halles,Paris,48.862567,2.345809
2,Quartier du Palais-Royal,Paris,48.863357,2.336062
3,Paris,Paris,48.859116,2.331839
4,Paris,Paris,48.859116,2.331839


## Rename columns

In [51]:
df.columns=['Neighbourhood','cities','Latitude','Longitude']
df.shape

(544, 4)

## Explore and cluster

### Create a map 

In [52]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [54]:
address = 'Paris'

geolocator = Nominatim(user_agent="Paris")
location = geolocator.geocode(address)
Latitude = location.latitude
Longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(Latitude, Longitude))

The geograpical coordinate of Toronto are 48.8566969, 2.3514616.


## Create a map of Paris, London and Madrid with neighborhoods

In [55]:
map_euro = folium.Map(location=[Latitude, Longitude], zoom_start=10)

# add markers to map
for lat, lng, cities, Neighbourhood in zip(df['Latitude'], df['Longitude'], df['cities'], df['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, cities)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_euro)  
    
map_euro

In [56]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
CLIENT_ID = 'YGGTUGIV2OW5MFKCKGTB141YOR24ODKPSQFVFVFG4Y1Q4EEG' # your Foursquare ID
CLIENT_SECRET = '1TTFU41DWEPLXE1UTZBLH5R3Y145UCZBGOCFLM53150DQTMD' # your Foursquare Secret
VERSION = '20200228' # Foursquare API version

radius=500
LIMIT=100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



Your credentails:
CLIENT_ID: YGGTUGIV2OW5MFKCKGTB141YOR24ODKPSQFVFVFG4Y1Q4EEG
CLIENT_SECRET:1TTFU41DWEPLXE1UTZBLH5R3Y145UCZBGOCFLM53150DQTMD


In [58]:
euro_venues = getNearbyVenues(names=df['Neighbourhood'], latitudes=df['Latitude'], longitudes=df['Longitude'])

Quartier Saint-Germain-l'Auxerrois
Les Halles
Quartier du Palais-Royal
Paris
Paris
Paris
Paris
Paris
Paris
Passage Sainte-Avoye
Quartier Saint-Merri
Paris
Paris
Notre-Dame
Quartier Saint-Victor
Paris
Paris
Quartier de la Sorbonne
Paris
Paris
Paris
Saint-Germain-des-Prés
Quartier Saint-Thomas-d'Aquin
Invalides
Paris
Paris
Champs-Élysées
Paris
Paris
Paris
Paris
Théâtre de la Porte-Saint-Martin
Paris
République Saint-Ambroise
Paris
Nation-Picpus
Paris
Bercy
Paris
Paris
Paris
Paris
Montparnasse
Petit-Montrouge
Paris
Paris
Grenelle
Javel
Auteuil
Paris
Paris
Paris
Paris
Batignolles
Épinettes
Paris
Goutte d'Or
La Chapelle
Paris
Paris
Paris
Belleville
Saint-Fargeau
Paris
Paris
La Acebeda
Alameda del Valle
Alcala de Henares
Alcorcón
Algete
Ambite
Aranjuez
Arroyomolinos
Batres
Belmonte de Tajo
Berzosa Del Lozoya
Boalo
Brea De Tajo
Buitrago
Cabanillas de la Sierra
Cadalso
Campo Real
Carabaña
Cenicientos
Cervera de Buitrago
Chinchón
Cobeña
Collado Villalba
Colmenar del Arroyo
Colmenarejo
Coslada
D

In [59]:
print(euro_venues.shape)
euro_venues.head()

(17987, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Quartier Saint-Germain-l'Auxerrois,48.860562,2.336489,Musée du Louvre,48.860847,2.336440,Art Museum
1,Quartier Saint-Germain-l'Auxerrois,48.860562,2.336489,Cour Carrée du Louvre,48.860360,2.338543,Pedestrian Plaza
2,Quartier Saint-Germain-l'Auxerrois,48.860562,2.336489,La Vénus de Milo (Vénus de Milo),48.859943,2.337234,Exhibit
3,Quartier Saint-Germain-l'Auxerrois,48.860562,2.336489,Cour Napoléon,48.861172,2.335088,Plaza
4,Quartier Saint-Germain-l'Auxerrois,48.860562,2.336489,Place du Palais Royal,48.862523,2.336688,Plaza


In [61]:
euro_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ajalvir,5,5,5,5,5,5
Alameda del Valle,4,4,4,4,4,4
Alcala de Henares,32,32,32,32,32,32
Alcobendas,9,9,9,9,9,9
Alcorcón,10,10,10,10,10,10
Aldea del Fresno,5,5,5,5,5,5
Algete,7,7,7,7,7,7
Alpedrete,4,4,4,4,4,4
Anchuelo,1,1,1,1,1,1
